# Utilization Prediction: Prediction of hourly utilization of the two sites

## Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os
os.environ["KERAS_BACKEND"] = "torch"

import keras
from keras import Sequential
from keras.layers import Dense

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

%matplotlib inline

## Load and prepare data

In [2]:
# Load data
charging_data = pd.read_pickle("data/charging_modified.pkl")
weather_data = pd.read_csv("data/weather_modified.csv")

# TODO: more filtering of features
# drop not important columns in weather data
weather_data.drop(columns = ["city", "cloud_cover_description", "pressure", "felt_temperature"], inplace = True)
weather_data["timestamp"] = pd.to_datetime(weather_data["timestamp"], utc = True).dt.floor(freq = "H")

# drop not important columns in charging data
charging_data.drop(columns = ["kWhPerMinute", "NoChargingTime", "user_kWhRequested", "user_modifiedAt", "doneChargingTime", "NoChargingTimeMinutes", "spaceID", "stationID", 
                              "user_userID", "ChargingTime", "user_paymentRequired_values"], inplace = True)

# Delete rows with null values
charging_data = charging_data.dropna()

# delete duplicate row index
charging_data.reset_index(drop = True, inplace = True)

charging_data.astype({
    "siteID": "category",
    "kWhDelivered": "int64",
    "ChargingTimeMinutes": "int64",
    "user_minutesAvailable": "int64",
    "user_milesRequested": "int64",
    "user_WhPerMile": "int64"
})

weather_data.astype({
    "temperature": "float",
    "cloud_cover": "float",
    "windspeed": "float",
    "precipitation": "float"
})

# check user_paymentRequired_values again?

# create dummy variables for user_paymentRequired_values column
charging_data = pd.get_dummies(charging_data, columns = ["siteID"], dtype = "int64")
charging_data.head()

,id,connectionTime,disconnectTime,kWhDelivered,userID,ChargingTimeMinutes,month,user_requestedDeparture,user_minutesAvailable,user_milesRequested,user_WhPerMile,siteID_1,siteID_2
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,25.016,194.0,262.683333,1,"Thu, 02 Jan 2020 20:51:54 GMT",463.0,100.0,250.0,1,0
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,33.097,4275.0,401.250000,1,"Thu, 02 Jan 2020 23:31:50 GMT",595.0,250.0,280.0,1,0
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,6.521,344.0,158.516667,1,"Thu, 02 Jan 2020 14:56:35 GMT",60.0,20.0,400.0,1,0
3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2.355,1117.0,78.783333,1,"Thu, 02 Jan 2020 15:04:58 GMT",65.0,20.0,400.0,1,0
4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,13.375,334.0,257.483333,1,"Thu, 02 Jan 2020 22:24:01 GMT",504.0,40.0,400.0,1,0


## Create feature vector X and labels Y

In [3]:
# split time interval between connectionTime and disconnectTime in minutes for every hour
def split_time_interval(index, start_date, end_date):
    indices = pd.DatetimeIndex([start_date])
    indices = indices.append(pd.date_range(start_date.ceil(freq = "H"), end_date.floor(freq = "H"), freq = "H"))
    indices = indices.append(pd.DatetimeIndex([end_date]))

    time_list = list(map(lambda x : 60 if x == 0 else x, indices.minute))
    time_list[0] = 60 - time_list[0]
    
    return list((time_list, indices))


# creates table with session id, charging time in minutes per hour and timestamps
def create_minute_table(id, minute_array):
    return list(zip([id] * len(minute_array[0]), minute_array[0], minute_array[1]))

In [4]:
# concat single minute lists to one big list with lists of tuples containing id, charging times in minutes per hour and timestamps
temp = []

for index in charging_data.index :
    charging_times = split_time_interval(charging_data.index[index], charging_data.loc[index, "connectionTime"], charging_data.loc[index, "disconnectTime"])
    temp.append(create_minute_table(charging_data.loc[index, "id"], charging_times))

# parse list of lists of triples into list of three lists: ids, charging time in minutes per hour and timestamps
ids, charging_times, timestamps = map(list, zip(*[tuple for list in temp for tuple in list]))

# create data.frame with three columns: session id, charging time and starting timestamp in minutes for every hour
charging_time_per_hour = pd.DataFrame(
                            {"id": ids,
                             "charging_time_per_hour": charging_times,
                             "timestamp": timestamps})
charging_time_per_hour.loc[:, "timestamp"] = charging_time_per_hour.loc[:, "timestamp"].dt.floor(freq = "H")

### Labels Y:

In [5]:
# roughly 50 EV charging stations per site * 60 minutes
maximum_utilization = 50 * 60

Y = list(map(lambda x : x / maximum_utilization, charging_time_per_hour.groupby("timestamp").sum().loc[:, "charging_time_per_hour"]))
Y = np.array(list(map(lambda x: 1.0 if x > 1.0 else x, Y)))
Y[1:20]

array([0.02      , 0.02      , 0.02      , 0.02      , 0.02      ,
       0.02      , 0.02      , 0.02      , 0.029     , 0.00733333,
       0.02      , 0.02      , 0.02      , 0.02      , 0.02      ,
       0.02      , 0.02      , 0.02      , 0.02      ])

### Feature Vector X:

In [6]:
# join charge time per hour with the rest of the charging data
X = charging_data.merge(charging_time_per_hour, how = "outer", on = "id").drop(["connectionTime", "disconnectTime", "user_requestedDeparture", "id", "userID", "charging_time_per_hour", "month"], axis = 1)

# group columns by timestamps
X = X.groupby("timestamp").agg({"kWhDelivered": "sum",
                                     "ChargingTimeMinutes": "sum",
                                     "user_minutesAvailable": "sum",
                                     "user_milesRequested": "sum",
                                     "user_WhPerMile": "mean"
                               })

# extract months from timestamp
X["month"] = list(map(lambda x: x.month, X.index))

# add weather data to feature vector
# TODO: find way to fix timestamp issue
X = X.merge(weather_data, how = "left", on = "timestamp").drop_duplicates("timestamp").drop(columns = ["timestamp"]).fillna(0).iloc[:, 1:3]
X

,ChargingTimeMinutes,user_minutesAvailable
0,560.033333,550.0
2,560.033333,550.0
3,560.033333,550.0
6,560.033333,550.0
8,560.033333,550.0
...,...,...
22173,542.716667,692.0
22174,542.716667,692.0
22175,542.716667,692.0
22176,542.716667,692.0


## Cross-validation: split data in training and test set

In [7]:
# Split data into training set and testing set nad normalize
scaler = StandardScaler()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= 0.3, random_state = 30)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
type(Y_train)

numpy.ndarray

## Prediction model 1: Neural Network

In [8]:
# Build the neural network
network = Sequential()

# Add layers
network.add(Dense(input_shape = (2,), units = 15, activation = "relu"))
network.add(Dense(units = 15, activation = "relu"))
network.add(Dense(units = 1, activation = "sigmoid"))

In [9]:
# Compiling the neural network
network.compile(optimizer = "adam", 
                   loss = "mean_squared_error", 
                   metrics = ["mean_squared_error"])

In [10]:
network.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Layer (type)                       ┃ Output Shape                  ┃     Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ dense (Dense)                      │ (None, 15)                    │          45 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ dense_1 (Dense)                    │ (None, 15)                    │         240 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ dense_2 (Dense)                    │ (None, 1)                     │          16 │
└────────────────────────────────────┴───────────────────────────────┴─────────────┘

 Total params: 301 (1.18 KB)

 Trainable params: 301 (1.18 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Fitting the Neural Network
network.fit(X_train, Y_train, batch_size=50, epochs=100)

Epoch 1/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0956 - mean_squared_error: 0.0956
Epoch 2/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 3/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0039 - mean_squared_error: 0.0039
Epoch 4/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0032 - mean_squared_error: 0.0032
Epoch 5/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0033 - mean_squared_error: 0.0033
Epoch 6/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0029 - mean_squared_error: 0.0029
Epoch 7/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0030 - mean_squared_error: 0.0030
Epoch 8/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0029 - mean_squared_error: 0.0029
Epoch 9/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0029 - mean_squared_error: 0.0029
Epoch 10/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0028 - mean_squared_error: 0.0028
Epoch 11/100
284/284 ━━━━━━━━

In [12]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error

mean_squared_error = mean_squared_error(Y_test, network.predict(X_test))
mean_absolute_error = mean_absolute_error(Y_test, network.predict(X_test))
mean_squared_log_error = mean_squared_log_error(Y_test, network.predict(X_test))
print(mean_squared_error)
print(mean_absolute_error)
print(mean_squared_log_error)

190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
0.002697558347476888
0.028930429960104295
0.0013353192003421727
